# Cleaning:
1. Remove admission_ids which have both icu_mortality and mortality_after_discharge as True
3. Remove row in each admission which are very close (<=5 hrs) to discharge. This is to avoid data leakage on discharge decision.
4. Add A
5. Add Y (Mortality within 7 days after discharge)
6. Add D (In-ICU mortality as a competing event)

In [42]:
import pandas as pd
import numpy as np

In [43]:
sf_mimicdata = pd.read_parquet('../Dataset/mimic-iv/from_pipeline/ltm_grided_clipped.parquet')

In [44]:
sf_mimicdata

,stay_id,grid_end,vent_mode__hours_since_last__last_12h,temperature__mean__last_12h,heart_rate__mean__last_12h,arterial_blood_pressure_mean__mean__last_12h,fluid_out_urine__mean__last_12h,pco2_arterial__mean__last_12h,respiratory_rate_measured__mean__last_12h,o2_saturation__mean__last_12h,...,raw_weight,unit_type,origin,los,intime,outtime,death_time_from_intime,icu_mortality,death_abs_time,mortality_after_discharge
0,mimic4-30000153,0 days 12:00:00,6.983333,37.625000,109.076920,84.380959,82.222221,44.0,17.230770,97.923080,...,73.0,Trauma,Emergency,1 days 15:17:10,2174-09-29 12:09:00,2174-10-01 03:26:10,NaT,0,NaT,0
1,mimic4-30000153,1 days 00:00:00,NaN,37.333332,104.416664,91.846153,55.000000,NaN,12.500000,95.250000,...,73.0,Trauma,Emergency,1 days 15:17:10,2174-09-29 12:09:00,2174-10-01 03:26:10,NaT,0,NaT,0
2,mimic4-30000153,1 days 12:00:00,NaN,37.569443,102.250000,95.476189,83.125000,NaN,14.272727,98.545456,...,73.0,Trauma,Emergency,1 days 15:17:10,2174-09-29 12:09:00,2174-10-01 03:26:10,NaT,0,NaT,0
3,mimic4-30000153,2 days 00:00:00,NaN,NaN,99.666664,91.333336,70.000000,NaN,15.333333,100.000000,...,73.0,Trauma,Emergency,1 days 15:17:10,2174-09-29 12:09:00,2174-10-01 03:26:10,NaT,0,NaT,0
4,mimic4-30000213,0 days 12:00:00,NaN,36.911114,78.384613,78.846153,324.166656,41.5,21.692308,98.846153,...,84.7,Medical,Other,1 days 15:14:48,2162-06-21 05:38:00,2162-06-22 20:52:48,1363 days 18:22:00,0,2166-03-16,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726804,mimic4-39999858,2 days 00:00:00,NaN,36.944447,64.833336,77.250000,NaN,NaN,21.500000,95.166664,...,123.8,Medical-Surgical,Direct Admit,3 days 21:35:50,2167-04-26 12:36:00,2167-04-30 10:11:50,NaT,0,NaT,0
726805,mimic4-39999858,2 days 12:00:00,NaN,36.902779,64.250000,81.818184,NaN,NaN,23.916666,92.550003,...,123.8,Medical-Surgical,Direct Admit,3 days 21:35:50,2167-04-26 12:36:00,2167-04-30 10:11:50,NaT,0,NaT,0
726806,mimic4-39999858,3 days 00:00:00,NaN,36.870373,68.166664,71.166664,NaN,NaN,24.500000,93.250000,...,123.8,Medical-Surgical,Direct Admit,3 days 21:35:50,2167-04-26 12:36:00,2167-04-30 10:11:50,NaT,0,NaT,0
726807,mimic4-39999858,3 days 12:00:00,NaN,36.722225,64.666664,69.666664,NaN,NaN,25.666666,92.166664,...,123.8,Medical-Surgical,Direct Admit,3 days 21:35:50,2167-04-26 12:36:00,2167-04-30 10:11:50,NaT,0,NaT,0


In [45]:
sf_mimicdata["timestep"] = (
    sf_mimicdata.sort_values(["stay_id", "grid_end"])
              .groupby("stay_id")
              .cumcount()
)


In [46]:
sf_mimicdata = sf_mimicdata.rename(columns={"timestep": "t"})

In [47]:
sf_mimicdata.columns

Index(['stay_id', 'grid_end', 'vent_mode__hours_since_last__last_12h',
       'temperature__mean__last_12h', 'heart_rate__mean__last_12h',
       'arterial_blood_pressure_mean__mean__last_12h',
       'fluid_out_urine__mean__last_12h', 'pco2_arterial__mean__last_12h',
       'respiratory_rate_measured__mean__last_12h',
       'o2_saturation__mean__last_12h', 'po2_arterial__mean__last_12h',
       'bicarbonate_arterial__last__last_12h',
       'activated_partial_thromboplastin_time__last__last_12h',
       'hemoglobin__last__last_12h', 'creatinine__last__last_12h',
       'ureum__last__last_12h', 'lactate__last__last_12h',
       'glasgow_coma_scale_total__last__last_12h', 'o2_flow__last__last_12h',
       'vent_mode__last__last_12h', 'raw_age', 'sex', 'raw_height',
       'raw_weight', 'unit_type', 'origin', 'los', 'intime', 'outtime',
       'death_time_from_intime', 'icu_mortality', 'death_abs_time',
       'mortality_after_discharge', 't'],
      dtype='object')

In [48]:
sf_mimicdata = sf_mimicdata.sort_values(by=['stay_id', 't'])

In [49]:
sf_mimicdata.stay_id.nunique()

89342

In [50]:
nan_columns = sf_mimicdata.columns[sf_mimicdata.isna().all()].tolist()
nan_columns

[]

In [51]:
#mimicdata = mimicdata.drop(columns=nan_columns)

In [52]:
sf_mimicdata.columns

Index(['stay_id', 'grid_end', 'vent_mode__hours_since_last__last_12h',
       'temperature__mean__last_12h', 'heart_rate__mean__last_12h',
       'arterial_blood_pressure_mean__mean__last_12h',
       'fluid_out_urine__mean__last_12h', 'pco2_arterial__mean__last_12h',
       'respiratory_rate_measured__mean__last_12h',
       'o2_saturation__mean__last_12h', 'po2_arterial__mean__last_12h',
       'bicarbonate_arterial__last__last_12h',
       'activated_partial_thromboplastin_time__last__last_12h',
       'hemoglobin__last__last_12h', 'creatinine__last__last_12h',
       'ureum__last__last_12h', 'lactate__last__last_12h',
       'glasgow_coma_scale_total__last__last_12h', 'o2_flow__last__last_12h',
       'vent_mode__last__last_12h', 'raw_age', 'sex', 'raw_height',
       'raw_weight', 'unit_type', 'origin', 'los', 'intime', 'outtime',
       'death_time_from_intime', 'icu_mortality', 'death_abs_time',
       'mortality_after_discharge', 't'],
      dtype='object')

In [53]:
sf_mimicdata.head(10)

,stay_id,grid_end,vent_mode__hours_since_last__last_12h,temperature__mean__last_12h,heart_rate__mean__last_12h,arterial_blood_pressure_mean__mean__last_12h,fluid_out_urine__mean__last_12h,pco2_arterial__mean__last_12h,respiratory_rate_measured__mean__last_12h,o2_saturation__mean__last_12h,...,unit_type,origin,los,intime,outtime,death_time_from_intime,icu_mortality,death_abs_time,mortality_after_discharge,t
0,mimic4-30000153,0 days 12:00:00,6.983333,37.625000,109.076920,84.380959,82.222221,44.0,17.230770,97.923080,...,Trauma,Emergency,1 days 15:17:10,2174-09-29 12:09:00,2174-10-01 03:26:10,NaT,0,NaT,0,0
1,mimic4-30000153,1 days 00:00:00,NaN,37.333332,104.416664,91.846153,55.000000,NaN,12.500000,95.250000,...,Trauma,Emergency,1 days 15:17:10,2174-09-29 12:09:00,2174-10-01 03:26:10,NaT,0,NaT,0,1
2,mimic4-30000153,1 days 12:00:00,NaN,37.569443,102.250000,95.476189,83.125000,NaN,14.272727,98.545456,...,Trauma,Emergency,1 days 15:17:10,2174-09-29 12:09:00,2174-10-01 03:26:10,NaT,0,NaT,0,2
3,mimic4-30000153,2 days 00:00:00,NaN,NaN,99.666664,91.333336,70.000000,NaN,15.333333,100.000000,...,Trauma,Emergency,1 days 15:17:10,2174-09-29 12:09:00,2174-10-01 03:26:10,NaT,0,NaT,0,3
4,mimic4-30000213,0 days 12:00:00,NaN,36.911114,78.384613,78.846153,324.166656,41.5,21.692308,98.846153,...,Medical,Other,1 days 15:14:48,2162-06-21 05:38:00,2162-06-22 20:52:48,1363 days 18:22:00,0,2166-03-16,0,0
5,mimic4-30000213,1 days 00:00:00,NaN,37.166668,85.250000,79.416664,230.000000,NaN,16.250000,99.083336,...,Medical,Other,1 days 15:14:48,2162-06-21 05:38:00,2162-06-22 20:52:48,1363 days 18:22:00,0,2166-03-16,0,1
6,mimic4-30000213,1 days 12:00:00,NaN,37.018520,85.083336,82.666664,183.333328,NaN,18.166666,96.000000,...,Medical,Other,1 days 15:14:48,2162-06-21 05:38:00,2162-06-22 20:52:48,1363 days 18:22:00,0,2166-03-16,0,2
7,mimic4-30000213,2 days 00:00:00,NaN,NaN,81.000000,86.000000,NaN,NaN,22.000000,96.000000,...,Medical,Other,1 days 15:14:48,2162-06-21 05:38:00,2162-06-22 20:52:48,1363 days 18:22:00,0,2166-03-16,0,3
8,mimic4-30000484,0 days 12:00:00,NaN,35.629631,89.166664,64.750000,114.000000,59.0,15.000000,99.750000,...,Medical,Emergency,2 days 11:29:35.999999,2136-01-14 17:23:32,2136-01-17 04:53:08,37 days 06:36:28,0,2136-02-21,0,0
9,mimic4-30000484,1 days 00:00:00,NaN,36.125000,90.750000,69.487183,71.111115,NaN,14.416667,99.833336,...,Medical,Emergency,2 days 11:29:35.999999,2136-01-14 17:23:32,2136-01-17 04:53:08,37 days 06:36:28,0,2136-02-21,0,1


In [54]:
sf_mimicdata.iloc[0]

stay_id                                                      mimic4-30000153
grid_end                                                     0 days 12:00:00
vent_mode__hours_since_last__last_12h                               6.983333
temperature__mean__last_12h                                           37.625
heart_rate__mean__last_12h                                         109.07692
arterial_blood_pressure_mean__mean__last_12h                       84.380959
fluid_out_urine__mean__last_12h                                    82.222221
pco2_arterial__mean__last_12h                                           44.0
respiratory_rate_measured__mean__last_12h                           17.23077
o2_saturation__mean__last_12h                                       97.92308
po2_arterial__mean__last_12h                                           233.0
bicarbonate_arterial__last__last_12h                                     NaN
activated_partial_thromboplastin_time__last__last_12h                    NaN

# Check if there are any stays with both icu_mortality=True and mortality_after_discharge=True

In [55]:
#ratio of mortality
last_rows = sf_mimicdata.groupby("stay_id").tail(1)
(last_rows["mortality_after_discharge"] == 1).mean()

0.033041570593897604

In [56]:
# Identify admission_ids where both icu_mortality and mortality_after_discharge are True. These are admissions which have death very close to discharge (<=12 hours)
# means that they were likely icu_mortality but were marked as mortality_after_discharge due to inconsistencies in recording?

# Identify stay_ids where BOTH outcomes are 1
conflicting_stays = sf_mimicdata.loc[
    (sf_mimicdata["icu_mortality"] == 1) &
    (sf_mimicdata["mortality_after_discharge"] == 1),
    "stay_id"
].unique()

# Set post-discharge mortality = 0 for those stay_ids
sf_mimicdata.loc[
    sf_mimicdata["stay_id"].isin(conflicting_stays),
    "mortality_after_discharge"
] = 0


In [57]:
len(conflicting_stays)

1917

In [58]:
#ratio of mortality
last_rows = sf_mimicdata.groupby("stay_id").tail(1)
(last_rows["mortality_after_discharge"] == 1).mean()

0.011584697007006784

In [59]:
sf_mimicdata.stay_id.nunique()

89342

sf_mimicdata[sf_mimicdata.stay_id==30423672][['intime', 'outtime', 't', 'LOS']]

pd.to_datetime('2176-10-23 12:18:47') + pd.to_timedelta((19+ 1) * 12, unit="h")

time_to_discharge = pd.to_datetime('2176-11-02 12:27:33') - (pd.to_datetime('2176-10-23 12:18:47') + pd.to_timedelta((18+1) * 12, unit="h"))
time_to_discharge

time_to_discharge > pd.Timedelta(hours=6)

# Checking: (Recorded) discharge timestamp is close to (<=5hrs) to observation timestamp
### Remove last row if time difference is less than or equal to 5 hours

In [60]:
sf_mimicdata = sf_mimicdata.sort_values(by=['stay_id', 't'])

In [61]:
# Parameters
BUCKET_HOURS = 12
DECISION_HORIZON_HOURS = 5  # drop rows whose bucket-end is within 5h of ICU discharge

# Ensure datetimes
intime  = pd.to_datetime(sf_mimicdata["intime"],  errors="coerce")
outtime = pd.to_datetime(sf_mimicdata["outtime"], errors="coerce")

# Reference timestamp for each row = end of the bucket:
# t=0 -> 12h, t=1 -> 24h, etc.
ref_time = intime + pd.to_timedelta((sf_mimicdata["t"] + 1) * BUCKET_HOURS, unit="h")
sf_mimicdata["ref_time"] = ref_time

# Time remaining until discharge (can be negative if ref_time > outtime)
time_to_discharge = outtime - ref_time

# Keep rows where:
#  - outtime is missing (can't evaluate; keep), OR
#  - more than DECISION_HORIZON_HOURS remain until discharge
mask_keep = time_to_discharge.isna() | (
    time_to_discharge > pd.Timedelta(hours=DECISION_HORIZON_HOURS)
)

# (Optional) quick audit before dropping
dropped = (~mask_keep).sum()
kept = mask_keep.sum()
print(f"Filtering by {DECISION_HORIZON_HOURS}h decision horizon: drop {dropped} rows, keep {kept} rows.")

# Apply filter
sf_mimicdata = sf_mimicdata[mask_keep].reset_index(drop=True)


Filtering by 5h decision horizon: drop 126729 rows, keep 600080 rows.


In [62]:
sf_mimicdata.stay_id.nunique()

85646

In [63]:
#ratio of mortality
last_rows = sf_mimicdata.groupby("stay_id").tail(1)
(last_rows["mortality_after_discharge"] == 1).mean()

0.011757700301239987

# Add A, Y and D

In [64]:
sf_mimicdata = sf_mimicdata.sort_values(by=['stay_id', 't'])

In [65]:
sf_mimicdata.loc[:, 'A'] = np.nan
sf_mimicdata.loc[:, 'D'] = 0        #[Disputed. Yet to think. Only when A=0 AND (icu_mortality occurs, D=1 or no icu mortality, D=0). If A=1, D=0 (is it undefined?np.nan)
sf_mimicdata.loc[:, 'Y'] = np.nan

In [66]:
# Fill in A
sf_mimicdata["A"] = 0

# Update "A" to 0 if "icu_mortality" is True, and A=1 in other case (~ icu_mortality) at the last row of each 'stay_id'
sf_mimicdata.loc[sf_mimicdata.groupby('stay_id').tail(1).index, 'A'] = sf_mimicdata.groupby('stay_id').tail(1)['icu_mortality'].apply(lambda x: 0 if x else 1)

In [67]:
# filter rows where icu_mortality = 1
subset = sf_mimicdata[sf_mimicdata["icu_mortality"] == 0]

In [68]:
# pick one at random
random_stay = subset.sample(1)  # remove random_state for true random
print(random_stay["stay_id"].iloc[0])

mimic4-32124823


In [69]:
sf_mimicdata[sf_mimicdata.stay_id=="mimic4-33598573"]

,stay_id,grid_end,vent_mode__hours_since_last__last_12h,temperature__mean__last_12h,heart_rate__mean__last_12h,arterial_blood_pressure_mean__mean__last_12h,fluid_out_urine__mean__last_12h,pco2_arterial__mean__last_12h,respiratory_rate_measured__mean__last_12h,o2_saturation__mean__last_12h,...,outtime,death_time_from_intime,icu_mortality,death_abs_time,mortality_after_discharge,t,ref_time,A,D,Y
218760,mimic4-33598573,0 days 12:00:00,NaN,36.833336,73.692307,97.000000,266.666656,53.0,17.153847,99.307693,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,0,2156-09-02 20:12:00,0,0,NaN
218761,mimic4-33598573,1 days 00:00:00,NaN,37.000000,70.583336,98.000000,191.875000,NaN,15.083333,99.416664,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,1,2156-09-03 08:12:00,0,0,NaN
218762,mimic4-33598573,1 days 12:00:00,NaN,37.277782,68.916664,108.666664,204.166672,30.0,13.500000,99.000000,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,2,2156-09-03 20:12:00,0,0,NaN
218763,mimic4-33598573,2 days 00:00:00,NaN,37.129627,69.083336,107.333336,80.000000,NaN,16.500000,99.750000,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,3,2156-09-04 08:12:00,0,0,NaN
218764,mimic4-33598573,2 days 12:00:00,NaN,37.185184,64.583336,104.833336,72.142860,NaN,11.333333,99.833336,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,4,2156-09-04 20:12:00,0,0,NaN
218765,mimic4-33598573,3 days 00:00:00,NaN,37.333332,70.000000,106.916664,133.571426,NaN,12.538462,99.461540,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,5,2156-09-05 08:12:00,0,0,NaN
218766,mimic4-33598573,3 days 12:00:00,NaN,36.851852,54.833332,93.454544,105.625000,NaN,13.833333,98.846153,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,6,2156-09-05 20:12:00,0,0,NaN
218767,mimic4-33598573,4 days 00:00:00,NaN,37.222225,47.500000,99.230766,77.428574,34.5,13.500000,98.583336,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,7,2156-09-06 08:12:00,0,0,NaN
218768,mimic4-33598573,4 days 12:00:00,NaN,36.759262,63.000000,95.416664,160.500000,NaN,15.500000,99.416664,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,8,2156-09-06 20:12:00,0,0,NaN
218769,mimic4-33598573,5 days 00:00:00,NaN,36.851852,72.833336,102.909088,213.636368,NaN,19.416666,99.916664,...,2156-09-10 19:57:09,10 days 15:48:00,0,2156-09-13,1,9,2156-09-07 08:12:00,0,0,NaN


In [70]:
# Fill in D and Y

# A = 0 (continued ICU care)
sf_mimicdata.loc[(sf_mimicdata['A'] == 0) & (sf_mimicdata['icu_mortality'] == False), ['Y', 'D']] = [np.nan, 0]

#sf_mimicdata.loc[(sf_mimicdata['A'] == 0) & (sf_mimicdata['icu_mortality'] == True),  ['Y', 'D']] = [np.nan, 1]
# Update "D" to 1 if "icu_mortality" is True at the last row of each 'stay_id'
sf_mimicdata.loc[sf_mimicdata.groupby('stay_id').tail(1).index, 'D'] = sf_mimicdata.groupby('stay_id').tail(1)['icu_mortality'].apply(lambda x: 1 if x else 0)

# A = 1 (discharged)
sf_mimicdata.loc[(sf_mimicdata['A'] == 1) & (sf_mimicdata['mortality_after_discharge'] == False), ['Y', 'D']] = [0, 0] #icu_mortality=0. Hence, D can be defined as 0?
sf_mimicdata.loc[(sf_mimicdata['A'] == 1) & (sf_mimicdata['mortality_after_discharge'] == True),  ['Y', 'D']] = [1, 0] # Same as above


In [71]:
sf_mimicdata

,stay_id,grid_end,vent_mode__hours_since_last__last_12h,temperature__mean__last_12h,heart_rate__mean__last_12h,arterial_blood_pressure_mean__mean__last_12h,fluid_out_urine__mean__last_12h,pco2_arterial__mean__last_12h,respiratory_rate_measured__mean__last_12h,o2_saturation__mean__last_12h,...,outtime,death_time_from_intime,icu_mortality,death_abs_time,mortality_after_discharge,t,ref_time,A,D,Y
0,mimic4-30000153,0 days 12:00:00,6.983333,37.625000,109.076920,84.380959,82.222221,44.0,17.230770,97.923080,...,2174-10-01 03:26:10,NaT,0,NaT,0,0,2174-09-30 00:09:00,0,0,NaN
1,mimic4-30000153,1 days 00:00:00,NaN,37.333332,104.416664,91.846153,55.000000,NaN,12.500000,95.250000,...,2174-10-01 03:26:10,NaT,0,NaT,0,1,2174-09-30 12:09:00,1,0,0.0
2,mimic4-30000213,0 days 12:00:00,NaN,36.911114,78.384613,78.846153,324.166656,41.5,21.692308,98.846153,...,2162-06-22 20:52:48,1363 days 18:22:00,0,2166-03-16,0,0,2162-06-21 17:38:00,0,0,NaN
3,mimic4-30000213,1 days 00:00:00,NaN,37.166668,85.250000,79.416664,230.000000,NaN,16.250000,99.083336,...,2162-06-22 20:52:48,1363 days 18:22:00,0,2166-03-16,0,1,2162-06-22 05:38:00,1,0,0.0
4,mimic4-30000484,0 days 12:00:00,NaN,35.629631,89.166664,64.750000,114.000000,59.0,15.000000,99.750000,...,2136-01-17 04:53:08,37 days 06:36:28,0,2136-02-21,0,0,2136-01-15 05:23:32,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600075,mimic4-39999858,1 days 12:00:00,NaN,36.777779,66.785713,77.250000,NaN,NaN,23.357143,93.071426,...,2167-04-30 10:11:50,NaT,0,NaT,0,2,2167-04-28 00:36:00,0,0,NaN
600076,mimic4-39999858,2 days 00:00:00,NaN,36.944447,64.833336,77.250000,NaN,NaN,21.500000,95.166664,...,2167-04-30 10:11:50,NaT,0,NaT,0,3,2167-04-28 12:36:00,0,0,NaN
600077,mimic4-39999858,2 days 12:00:00,NaN,36.902779,64.250000,81.818184,NaN,NaN,23.916666,92.550003,...,2167-04-30 10:11:50,NaT,0,NaT,0,4,2167-04-29 00:36:00,0,0,NaN
600078,mimic4-39999858,3 days 00:00:00,NaN,36.870373,68.166664,71.166664,NaN,NaN,24.500000,93.250000,...,2167-04-30 10:11:50,NaT,0,NaT,0,5,2167-04-29 12:36:00,0,0,NaN


sf_mimicdata['all_M'] = ((sf_mimicdata['mortality_within_7days_after_discharge'].fillna(0) == 1) | (sf_mimicdata['icu_mortality'].fillna(0) == 1)).astype(int)

In [72]:
sf_mimicdata = sf_mimicdata.reset_index(drop=True)

sf_mimicdata = sf_mimicdata.rename(columns={
    'Y': 'Y_old',
    'all_M': 'Y'
})

In [73]:
sf_mimicdata.groupby('stay_id').tail(1)['Y'].mean()

0.013269555133881509

In [74]:
sf_mimicdata.groupby('stay_id').tail(1)['D'].mean()

0.11393410083366415

In [75]:
sf_mimicdata.to_parquet('../Dataset/mimic-iv/from_pipeline/ltm_grided_clipped_A_Y_D_SelfPipeline.parquet')

In [76]:
sf_mimicdata.columns

Index(['stay_id', 'grid_end', 'vent_mode__hours_since_last__last_12h',
       'temperature__mean__last_12h', 'heart_rate__mean__last_12h',
       'arterial_blood_pressure_mean__mean__last_12h',
       'fluid_out_urine__mean__last_12h', 'pco2_arterial__mean__last_12h',
       'respiratory_rate_measured__mean__last_12h',
       'o2_saturation__mean__last_12h', 'po2_arterial__mean__last_12h',
       'bicarbonate_arterial__last__last_12h',
       'activated_partial_thromboplastin_time__last__last_12h',
       'hemoglobin__last__last_12h', 'creatinine__last__last_12h',
       'ureum__last__last_12h', 'lactate__last__last_12h',
       'glasgow_coma_scale_total__last__last_12h', 'o2_flow__last__last_12h',
       'vent_mode__last__last_12h', 'raw_age', 'sex', 'raw_height',
       'raw_weight', 'unit_type', 'origin', 'los', 'intime', 'outtime',
       'death_time_from_intime', 'icu_mortality', 'death_abs_time',
       'mortality_after_discharge', 't', 'ref_time', 'A', 'D', 'Y'],
      dtype='ob